In [3]:
"""
PAPER 3 - PHASE 1.1: DISTRIBUTIONAL ANALYSIS
============================================
Deep dive into feature distributions to understand how each variable
discriminates between Panic Disorder and Normal cases.

Author: Generated for Panic Disorder ML Investigation
Date: 2025
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 11

# Paths
DATA_PATH = '/Users/filipecarvalho/Documents/data_science_projects/Panic.3/NHANES_panic_12features.csv'
OUTPUT_DIR = Path('/Users/filipecarvalho/Documents/data_science_projects/Panic.3/results/phase1_distributions')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Feature names with descriptions
FEATURE_INFO = {
    'BIX_BIDFAT': 'Body Fat Mass (kg)',
    'DEMO_INDFMPIR': 'Poverty Income Ratio',
    'DEMO_RIDAGEMN': 'Age (months)',
    'DEMO_RIAGENDR': 'Gender',
    'BPX_BPXDAR': 'Diastolic Blood Pressure',
    'WHQ_WHD050': 'Weight 1 Year Ago',
    'DEMO_DMDHHSIZ': 'Household Size',
    'BMX_BMXBMI': 'Body Mass Index',
    'ALQ_ALQ130': 'Alcohol Consumption',
    'MCQ_MCQ250F': 'Family History HTN/Stroke',
    'MPQ_MPQ070': 'Lower Back Pain',
    'DEMO_DMDMARTL': 'Marital Status'
}

# Biological grouping for theoretical analysis
BIOLOGICAL_DOMAINS = {
    'Biological': ['BPX_BPXDAR', 'BMX_BMXBMI', 'BIX_BIDFAT', 'WHQ_WHD050'],
    'Psychological': ['MPQ_MPQ070', 'MCQ_MCQ250F'],
    'Social': ['DEMO_INDFMPIR', 'DEMO_DMDMARTL', 'DEMO_RIAGENDR', 
               'DEMO_RIDAGEMN', 'DEMO_DMDHHSIZ', 'ALQ_ALQ130']
}

print("="*80)
print("🔬 PAPER 3 - PHASE 1.1: DISTRIBUTIONAL ANALYSIS")
print("="*80)
print(f"\n📂 Output directory: {OUTPUT_DIR}")

# ==================== LOAD DATA ====================
print("\n📊 Loading data...")
df = pd.read_csv(DATA_PATH)
print(f"   ✅ Loaded {len(df)} samples with {df.shape[1]} columns")

# Check for missing values
missing_summary = df.isnull().sum()
if missing_summary.sum() > 0:
    print("\n⚠️  Missing values detected:")
    for col in missing_summary[missing_summary > 0].index:
        pct = (missing_summary[col] / len(df)) * 100
        print(f"   {col}: {missing_summary[col]} ({pct:.2f}%)")
    print("\n   Strategy: Will drop rows with any missing values for clean analysis")
    df_clean = df.dropna()
    print(f"   ✅ Clean dataset: {len(df_clean)} samples ({len(df)-len(df_clean)} dropped)")
else:
    df_clean = df.copy()
    print("   ✅ No missing values!")

# Separate by group
df_pd = df_clean[df_clean['target'] == 1]
df_normal = df_clean[df_clean['target'] == 0]

print(f"\n🎯 Sample sizes:")
print(f"   Panic Disorder: {len(df_pd)} ({len(df_pd)/len(df_clean)*100:.2f}%)")
print(f"   Normal:         {len(df_normal)} ({len(df_normal)/len(df_clean)*100:.2f}%)")

features = list(FEATURE_INFO.keys())

# ==================== STATISTICAL ANALYSIS ====================
print("\n" + "="*80)
print("📈 COMPUTING STATISTICAL METRICS")
print("="*80)

results = []

for feat in features:
    print(f"\n🔍 Analyzing: {feat}")
    
    # Get data
    pd_vals = df_pd[feat].values
    normal_vals = df_normal[feat].values
    
    # Basic statistics
    pd_mean = np.mean(pd_vals)
    pd_std = np.std(pd_vals, ddof=1)
    pd_median = np.median(pd_vals)
    pd_min = np.min(pd_vals)
    pd_max = np.max(pd_vals)
    
    normal_mean = np.mean(normal_vals)
    normal_std = np.std(normal_vals, ddof=1)
    normal_median = np.median(normal_vals)
    normal_min = np.min(normal_vals)
    normal_max = np.max(normal_vals)
    
    # T-test
    t_stat, t_pval = stats.ttest_ind(pd_vals, normal_vals)
    
    # Kolmogorov-Smirnov test
    ks_stat, ks_pval = stats.ks_2samp(pd_vals, normal_vals)
    
    # Cohen's d (effect size)
    pooled_std = np.sqrt(((len(pd_vals)-1)*pd_std**2 + (len(normal_vals)-1)*normal_std**2) / 
                         (len(pd_vals) + len(normal_vals) - 2))
    cohens_d = (pd_mean - normal_mean) / pooled_std
    
    # Effect size interpretation
    if abs(cohens_d) < 0.2:
        effect_size_interp = "Negligible"
    elif abs(cohens_d) < 0.5:
        effect_size_interp = "Small"
    elif abs(cohens_d) < 0.8:
        effect_size_interp = "Medium"
    else:
        effect_size_interp = "Large"
    
    # Overlap coefficient (approximate using normal distributions)
    # Area under the minimum of two normal distributions
    def overlap_coefficient(mu1, sigma1, n1, mu2, sigma2, n2):
        """Calculate overlap coefficient between two distributions"""
        # Create range for integration
        x_min = min(mu1 - 4*sigma1, mu2 - 4*sigma2)
        x_max = max(mu1 + 4*sigma1, mu2 + 4*sigma2)
        x = np.linspace(x_min, x_max, 10000)
        
        # PDF for both distributions
        pdf1 = stats.norm.pdf(x, mu1, sigma1)
        pdf2 = stats.norm.pdf(x, mu2, sigma2)
        
        # Overlap is integral of minimum
        overlap = np.trapz(np.minimum(pdf1, pdf2), x)
        return overlap
    
    overlap = overlap_coefficient(pd_mean, pd_std, len(pd_vals), 
                                  normal_mean, normal_std, len(normal_vals))
    
    # Mann-Whitney U test (non-parametric alternative)
    u_stat, u_pval = stats.mannwhitneyu(pd_vals, normal_vals, alternative='two-sided')
    
    print(f"   PD Mean: {pd_mean:.3f} ± {pd_std:.3f}")
    print(f"   Normal Mean: {normal_mean:.3f} ± {normal_std:.3f}")
    print(f"   Cohen's d: {cohens_d:.3f} ({effect_size_interp})")
    print(f"   Overlap: {overlap:.3f} ({(1-overlap)*100:.1f}% separation)")
    print(f"   T-test p-value: {t_pval:.2e}")
    print(f"   KS-test p-value: {ks_pval:.2e}")
    
    results.append({
        'Feature': feat,
        'Description': FEATURE_INFO[feat],
        'PD_Mean': pd_mean,
        'PD_Std': pd_std,
        'PD_Median': pd_median,
        'PD_Min': pd_min,
        'PD_Max': pd_max,
        'Normal_Mean': normal_mean,
        'Normal_Std': normal_std,
        'Normal_Median': normal_median,
        'Normal_Min': normal_min,
        'Normal_Max': normal_max,
        'Mean_Diff': pd_mean - normal_mean,
        'Cohens_d': cohens_d,
        'Effect_Size': effect_size_interp,
        'Overlap_Coef': overlap,
        'Separation_Pct': (1 - overlap) * 100,
        'T_Statistic': t_stat,
        'T_Pvalue': t_pval,
        'KS_Statistic': ks_stat,
        'KS_Pvalue': ks_pval,
        'U_Statistic': u_stat,
        'U_Pvalue': u_pval
    })

# Create results DataFrame
df_results = pd.DataFrame(results)

# Add absolute value column for sorting and analysis
df_results['Cohens_d_abs'] = df_results['Cohens_d'].abs()

# ==================== SAVE STATISTICAL RESULTS ====================
print("\n" + "="*80)
print("💾 SAVING STATISTICAL RESULTS")
print("="*80)

# Full results table
output_file = OUTPUT_DIR / 'statistical_analysis_complete.csv'
df_results.to_csv(output_file, index=False)
print(f"\n✅ Full results saved to: {output_file}")

# Summary table (for paper)
summary_cols = ['Feature', 'Description', 'PD_Mean', 'PD_Std', 'Normal_Mean', 
                'Normal_Std', 'Cohens_d', 'Effect_Size', 'Separation_Pct', 'T_Pvalue']
df_summary = df_results[summary_cols].copy()
df_summary = df_summary.round({
    'PD_Mean': 2, 'PD_Std': 2, 'Normal_Mean': 2, 'Normal_Std': 2,
    'Cohens_d': 3, 'Separation_Pct': 1
})
df_summary['T_Pvalue'] = df_summary['T_Pvalue'].apply(lambda x: f'{x:.2e}')

output_file_summary = OUTPUT_DIR / 'Table1_descriptive_statistics.csv'
df_summary.to_csv(output_file_summary, index=False)
print(f"✅ Summary table saved to: {output_file_summary}")

# ==================== VISUALIZATIONS ====================
print("\n" + "="*80)
print("🎨 CREATING VISUALIZATIONS")
print("="*80)

# 1. VIOLIN PLOTS - All features in one figure
print("\n📊 Creating comprehensive violin plots...")
fig, axes = plt.subplots(4, 3, figsize=(20, 16))
axes = axes.flatten()

for idx, feat in enumerate(features):
    ax = axes[idx]
    
    # Prepare data for violin plot
    plot_data = pd.DataFrame({
        'Value': np.concatenate([df_pd[feat].values, df_normal[feat].values]),
        'Group': ['Panic Disorder']*len(df_pd) + ['Normal']*len(df_normal)
    })
    
    # Create violin plot
    parts = ax.violinplot([df_pd[feat].values, df_normal[feat].values],
                          positions=[0, 1],
                          showmeans=True,
                          showextrema=True,
                          showmedians=True)
    
    # Color the violins
    colors = ['#FF6B6B', '#4ECDC4']  # Red for PD, Teal for Normal
    for pc, color in zip(parts['bodies'], colors):
        pc.set_facecolor(color)
        pc.set_alpha(0.7)
    
    # Add individual points with jitter (sample if too many)
    np.random.seed(42)
    
    # PD points
    pd_sample = df_pd[feat].values
    if len(pd_sample) > 100:
        pd_sample = np.random.choice(pd_sample, 100, replace=False)
    jitter_pd = np.random.normal(0, 0.04, size=len(pd_sample))
    ax.scatter(jitter_pd, pd_sample, alpha=0.3, s=20, color='#FF6B6B', edgecolors='black', linewidths=0.5)
    
    # Normal points (sample more heavily)
    normal_sample = df_normal[feat].values
    if len(normal_sample) > 200:
        normal_sample = np.random.choice(normal_sample, 200, replace=False)
    jitter_normal = np.random.normal(1, 0.04, size=len(normal_sample))
    ax.scatter(jitter_normal, normal_sample, alpha=0.2, s=10, color='#4ECDC4', edgecolors='black', linewidths=0.3)
    
    # Styling
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['PD', 'Normal'])
    ax.set_title(f'{FEATURE_INFO[feat]}\n(Cohen\'s d = {df_results.iloc[idx]["Cohens_d"]:.3f})', 
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Value', fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add separation info
    separation = df_results.iloc[idx]['Separation_Pct']
    effect = df_results.iloc[idx]['Effect_Size']
    ax.text(0.5, ax.get_ylim()[1]*0.95, f'{separation:.1f}% sep. ({effect})',
            ha='center', va='top', fontsize=9, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Feature Distributions: Panic Disorder vs Normal\n(Phase 1.1 - Distributional Analysis)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure1_violin_plots_all_features.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Violin plots saved to: {output_fig}")
plt.close()

# 2. EFFECT SIZES RANKED
print("\n📊 Creating effect sizes ranking plot...")
df_sorted = df_results.sort_values('Cohens_d_abs', ascending=True)

fig, ax = plt.subplots(figsize=(12, 8))
colors_bar = ['#FF6B6B' if x > 0 else '#4ECDC4' for x in df_sorted['Cohens_d']]
bars = ax.barh(range(len(df_sorted)), df_sorted['Cohens_d'], color=colors_bar, alpha=0.7, edgecolor='black')

# Add value labels
for i, (val, effect) in enumerate(zip(df_sorted['Cohens_d'], df_sorted['Effect_Size'])):
    ax.text(val + 0.02 if val > 0 else val - 0.02, i, f'{val:.3f} ({effect})', 
            va='center', ha='left' if val > 0 else 'right', fontsize=9)

ax.set_yticks(range(len(df_sorted)))
ax.set_yticklabels([FEATURE_INFO[feat] for feat in df_sorted['Feature']], fontsize=10)
ax.set_xlabel("Cohen's d (Effect Size)", fontsize=12, fontweight='bold')
ax.set_title("Effect Sizes: Panic Disorder vs Normal\n(Sorted by Magnitude)", 
             fontsize=14, fontweight='bold')
ax.axvline(0, color='black', linewidth=1, linestyle='--')
ax.grid(True, alpha=0.3, axis='x')

# Add reference lines for effect size interpretation
ax.axvline(0.2, color='green', linewidth=0.5, linestyle=':', alpha=0.5, label='Small (0.2)')
ax.axvline(0.5, color='orange', linewidth=0.5, linestyle=':', alpha=0.5, label='Medium (0.5)')
ax.axvline(0.8, color='red', linewidth=0.5, linestyle=':', alpha=0.5, label='Large (0.8)')
ax.axvline(-0.2, color='green', linewidth=0.5, linestyle=':', alpha=0.5)
ax.axvline(-0.5, color='orange', linewidth=0.5, linestyle=':', alpha=0.5)
ax.axvline(-0.8, color='red', linewidth=0.5, linestyle=':', alpha=0.5)
ax.legend(loc='lower right', fontsize=9)

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure2_effect_sizes_ranked.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Effect sizes plot saved to: {output_fig}")
plt.close()

# 3. SEPARATION HEATMAP
print("\n📊 Creating separation heatmap...")
fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data for heatmap
heatmap_data = df_results.set_index('Feature')[['Separation_Pct']].T
heatmap_data.index = ['% Separation']

# Create heatmap
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', 
            cbar_kws={'label': '% Separation'}, ax=ax, 
            vmin=0, vmax=100, linewidths=1, linecolor='black')

ax.set_xticklabels([FEATURE_INFO[feat] for feat in heatmap_data.columns], rotation=45, ha='right')
ax.set_title('Feature Separation Power: Panic Disorder vs Normal\n(Higher = Better Discrimination)', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure3_separation_heatmap.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Separation heatmap saved to: {output_fig}")
plt.close()

# 4. OVERLAPPING DISTRIBUTIONS (Top 4 features)
print("\n📊 Creating overlapping distribution plots (top 4 features by effect size)...")
top4_features = df_results.nlargest(4, 'Cohens_d_abs')['Feature'].tolist()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, feat in enumerate(top4_features):
    ax = axes[idx]
    
    # Create histograms with KDE
    ax.hist(df_normal[feat], bins=50, alpha=0.5, color='#4ECDC4', label='Normal', density=True, edgecolor='black')
    ax.hist(df_pd[feat], bins=30, alpha=0.5, color='#FF6B6B', label='Panic Disorder', density=True, edgecolor='black')
    
    # Add KDE
    from scipy.stats import gaussian_kde
    
    # Normal KDE
    kde_normal = gaussian_kde(df_normal[feat].values)
    x_normal = np.linspace(df_normal[feat].min(), df_normal[feat].max(), 1000)
    ax.plot(x_normal, kde_normal(x_normal), color='#4ECDC4', linewidth=2.5, label='Normal KDE')
    
    # PD KDE
    kde_pd = gaussian_kde(df_pd[feat].values)
    x_pd = np.linspace(df_pd[feat].min(), df_pd[feat].max(), 1000)
    ax.plot(x_pd, kde_pd(x_pd), color='#FF6B6B', linewidth=2.5, label='PD KDE')
    
    # Styling
    ax.set_xlabel('Value', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    feat_info = df_results[df_results['Feature'] == feat].iloc[0]
    ax.set_title(f'{FEATURE_INFO[feat]}\nCohen\'s d = {feat_info["Cohens_d"]:.3f}, Overlap = {feat_info["Overlap_Coef"]:.3f}',
                 fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.suptitle('Distribution Overlap Analysis: Top 4 Features by Effect Size\n(Phase 1.1)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure4_distribution_overlaps_top4.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Distribution overlaps saved to: {output_fig}")
plt.close()

# ==================== DOMAIN ANALYSIS ====================
print("\n" + "="*80)
print("🧬 BIOLOGICAL DOMAIN ANALYSIS")
print("="*80)

domain_results = []

for domain_name, domain_features in BIOLOGICAL_DOMAINS.items():
    print(f"\n🔍 Analyzing domain: {domain_name}")
    
    # Get effect sizes for features in this domain
    domain_effects = df_results[df_results['Feature'].isin(domain_features)]['Cohens_d'].abs()
    
    print(f"   Features: {len(domain_features)}")
    print(f"   Mean |Cohen's d|: {domain_effects.mean():.3f}")
    print(f"   Max |Cohen's d|: {domain_effects.max():.3f}")
    print(f"   Features: {[FEATURE_INFO[f] for f in domain_features]}")
    
    domain_results.append({
        'Domain': domain_name,
        'N_Features': len(domain_features),
        'Mean_Effect_Size': domain_effects.mean(),
        'Max_Effect_Size': domain_effects.max(),
        'Min_Effect_Size': domain_effects.min(),
        'Features': ', '.join([FEATURE_INFO[f] for f in domain_features])
    })

df_domains = pd.DataFrame(domain_results)
output_file_domains = OUTPUT_DIR / 'Table2_domain_analysis.csv'
df_domains.to_csv(output_file_domains, index=False)
print(f"\n✅ Domain analysis saved to: {output_file_domains}")

# Domain comparison plot
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(BIOLOGICAL_DOMAINS))
means = [df_domains[df_domains['Domain'] == d]['Mean_Effect_Size'].values[0] for d in BIOLOGICAL_DOMAINS.keys()]
maxs = [df_domains[df_domains['Domain'] == d]['Max_Effect_Size'].values[0] for d in BIOLOGICAL_DOMAINS.keys()]

width = 0.35
bars1 = ax.bar(x - width/2, means, width, label='Mean |Cohen\'s d|', color='#4ECDC4', edgecolor='black')
bars2 = ax.bar(x + width/2, maxs, width, label='Max |Cohen\'s d|', color='#FF6B6B', edgecolor='black')

ax.set_ylabel('Effect Size (|Cohen\'s d|)', fontsize=12, fontweight='bold')
ax.set_title('Feature Discrimination by Biological Domain', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(BIOLOGICAL_DOMAINS.keys(), fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure5_domain_comparison.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Domain comparison plot saved to: {output_fig}")
plt.close()

# ==================== SUMMARY REPORT ====================
print("\n" + "="*80)
print("📝 GENERATING SUMMARY REPORT")
print("="*80)

summary_report = f"""
PHASE 1.1: DISTRIBUTIONAL ANALYSIS - SUMMARY REPORT
====================================================

Dataset Information:
-------------------
- Total samples: {len(df_clean)}
- Panic Disorder cases: {len(df_pd)} ({len(df_pd)/len(df_clean)*100:.2f}%)
- Normal cases: {len(df_normal)} ({len(df_normal)/len(df_clean)*100:.2f}%)
- Number of features analyzed: {len(features)}

Key Findings:
-------------

1. EFFECT SIZES (Cohen's d):
   - Mean |Cohen's d| across all features: {df_results['Cohens_d'].abs().mean():.3f}
   - Largest effect size: {df_results.loc[df_results['Cohens_d_abs'].idxmax(), 'Feature']} 
     ({FEATURE_INFO[df_results.loc[df_results['Cohens_d_abs'].idxmax(), 'Feature']]})
     Cohen's d = {df_results['Cohens_d_abs'].max():.3f}
   - Number of features with Large effect (|d| > 0.8): {len(df_results[df_results['Cohens_d_abs'] > 0.8])}
   - Number of features with Medium effect (0.5 < |d| < 0.8): {len(df_results[(df_results['Cohens_d_abs'] > 0.5) & (df_results['Cohens_d_abs'] <= 0.8)])}
   - Number of features with Small effect (0.2 < |d| < 0.5): {len(df_results[(df_results['Cohens_d_abs'] > 0.2) & (df_results['Cohens_d_abs'] <= 0.5)])}

2. DISTRIBUTION SEPARATION:
   - Mean separation across features: {df_results['Separation_Pct'].mean():.1f}%
   - Best separating feature: {df_results.loc[df_results['Separation_Pct'].idxmax(), 'Feature']}
     ({FEATURE_INFO[df_results.loc[df_results['Separation_Pct'].idxmax(), 'Feature']]})
     Separation = {df_results['Separation_Pct'].max():.1f}%
   - Features with >50% separation: {len(df_results[df_results['Separation_Pct'] > 50])}

3. STATISTICAL SIGNIFICANCE:
   - All features significant (p < 0.05): {len(df_results[df_results['T_Pvalue'] < 0.05]) == len(features)}
   - Features with p < 0.001: {len(df_results[df_results['T_Pvalue'] < 0.001])}

4. BIOLOGICAL DOMAIN ANALYSIS:
   - Biological domain mean effect: {df_domains[df_domains['Domain'] == 'Biological']['Mean_Effect_Size'].values[0]:.3f}
   - Psychological domain mean effect: {df_domains[df_domains['Domain'] == 'Psychological']['Mean_Effect_Size'].values[0]:.3f}
   - Social domain mean effect: {df_domains[df_domains['Domain'] == 'Social']['Mean_Effect_Size'].values[0]:.3f}

Top 5 Features by Discrimination Power:
---------------------------------------
"""

top5 = df_results.nlargest(5, 'Cohens_d_abs')
for i, row in enumerate(top5.itertuples(), 1):
    summary_report += f"\n{i}. {FEATURE_INFO[row.Feature]}"
    summary_report += f"\n   Cohen's d: {row.Cohens_d:.3f} ({row.Effect_Size})"
    summary_report += f"\n   Separation: {row.Separation_Pct:.1f}%"
    summary_report += f"\n   PD Mean: {row.PD_Mean:.2f} ± {row.PD_Std:.2f}"
    summary_report += f"\n   Normal Mean: {row.Normal_Mean:.2f} ± {row.Normal_Std:.2f}"

summary_report += f"""

Interpretation:
---------------
The distributional analysis reveals that while individual features show varying 
degrees of discrimination between Panic Disorder and Normal cases, NO SINGLE 
FEATURE achieves perfect separation. This suggests that the 100% accuracy 
observed in the full model is likely due to SYNERGISTIC INTERACTIONS between 
multiple features rather than the dominance of any single variable.

The biological domain shows {"stronger" if df_domains[df_domains['Domain'] == 'Biological']['Mean_Effect_Size'].values[0] > df_domains['Mean_Effect_Size'].mean() else "comparable"} 
discrimination compared to psychological and social domains, suggesting that 
physiological markers may play an important role in the classification.

Next Steps:
-----------
1. Investigate feature interactions (SHAP analysis)
2. Examine decision boundaries using decision trees
3. Explore phenotypic clustering within PD cases
4. Analyze multivariate patterns using dimensionality reduction

Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Save summary report
output_report = OUTPUT_DIR / 'PHASE1_SUMMARY_REPORT.txt'
with open(output_report, 'w') as f:
    f.write(summary_report)
print(f"\n✅ Summary report saved to: {output_report}")

print("\n" + "="*80)
print("✅ PHASE 1.1 COMPLETE!")
print("="*80)
print(f"\n📂 All results saved to: {OUTPUT_DIR}")
print("\n📊 Generated outputs:")
print("   - statistical_analysis_complete.csv")
print("   - Table1_descriptive_statistics.csv")
print("   - Table2_domain_analysis.csv")
print("   - Figure1_violin_plots_all_features.png")
print("   - Figure2_effect_sizes_ranked.png")
print("   - Figure3_separation_heatmap.png")
print("   - Figure4_distribution_overlaps_top4.png")
print("   - Figure5_domain_comparison.png")
print("   - PHASE1_SUMMARY_REPORT.txt")
print("\n🚀 Ready for Phase 1.2: Multidimensional Visualization!")

🔬 PAPER 3 - PHASE 1.1: DISTRIBUTIONAL ANALYSIS

📂 Output directory: /Users/filipecarvalho/Documents/data_science_projects/Panic.3/results/phase1_distributions

📊 Loading data...
   ✅ Loaded 6581 samples with 13 columns

⚠️  Missing values detected:
   BIX_BIDFAT: 1901 (28.89%)
   DEMO_INDFMPIR: 423 (6.43%)
   BPX_BPXDAR: 429 (6.52%)
   WHQ_WHD050: 4 (0.06%)
   BMX_BMXBMI: 233 (3.54%)
   ALQ_ALQ130: 1917 (29.13%)
   DEMO_DMDMARTL: 176 (2.67%)

   Strategy: Will drop rows with any missing values for clean analysis
   ✅ Clean dataset: 3279 samples (3302 dropped)

🎯 Sample sizes:
   Panic Disorder: 115 (3.51%)
   Normal:         3164 (96.49%)

📈 COMPUTING STATISTICAL METRICS

🔍 Analyzing: BIX_BIDFAT
   PD Mean: 26.254 ± 13.685
   Normal Mean: 22.911 ± 11.037
   Cohen's d: 0.300 (Small)
   Overlap: 0.858 (14.2% separation)
   T-test p-value: 1.59e-03
   KS-test p-value: 2.51e-11

🔍 Analyzing: DEMO_INDFMPIR
   PD Mean: 2.063 ± 1.474
   Normal Mean: 2.913 ± 1.608
   Cohen's d: -0.531 (Medium)

In [4]:
"""
DEEP INVESTIGATION: WHQ_WHD050 (Weight 1 Year Ago)
===================================================
This feature shows 98.5% separation but moderate Cohen's d.
Something unusual is happening - let's find out what!

Author: Panic Disorder ML Investigation
Date: 2025-11-11
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
OUTPUT_DIR = Path('/Users/filipecarvalho/Documents/data_science_projects/Panic.3/results/investigation_WHD050')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("="*80)
print("🔍 DEEP INVESTIGATION: WHQ_WHD050 (Weight 1 Year Ago)")
print("="*80)

# ==================== LOAD BOTH DATASETS ====================
print("\n📊 Loading datasets...")

# Full dataset (with missing values)
df_full = pd.read_csv('/Users/filipecarvalho/Documents/data_science_projects/Panic.3/NHANES_panic_12features.csv')
print(f"   Full dataset: {len(df_full)} samples")

# Clean dataset (used in Phase 1)
df_clean = df_full.dropna()
print(f"   Clean dataset: {len(df_clean)} samples")
print(f"   Dropped: {len(df_full) - len(df_clean)} samples ({(len(df_full) - len(df_clean))/len(df_full)*100:.1f}%)")

feature = 'WHQ_WHD050'

# ==================== MISSING VALUE ANALYSIS ====================
print("\n" + "="*80)
print("🔍 MISSING VALUE ANALYSIS")
print("="*80)

missing_full = df_full[feature].isnull().sum()
missing_pct = (missing_full / len(df_full)) * 100

print(f"\nTotal missing values: {missing_full} ({missing_pct:.2f}%)")

# Missing by group
missing_pd = df_full[df_full['target'] == 1][feature].isnull().sum()
missing_normal = df_full[df_full['target'] == 0][feature].isnull().sum()
total_pd = len(df_full[df_full['target'] == 1])
total_normal = len(df_full[df_full['target'] == 0])

print(f"\nMissing in PD group: {missing_pd}/{total_pd} ({missing_pd/total_pd*100:.2f}%)")
print(f"Missing in Normal group: {missing_normal}/{total_normal} ({missing_normal/total_normal*100:.2f}%)")

if missing_pd/total_pd > missing_normal/total_normal * 1.5:
    print("⚠️  WARNING: Much higher missingness in PD group!")
elif missing_normal/total_normal > missing_pd/total_pd * 1.5:
    print("⚠️  WARNING: Much higher missingness in Normal group!")

# ==================== BASIC STATISTICS ====================
print("\n" + "="*80)
print("📊 BASIC STATISTICS (Clean Data)")
print("="*80)

df_pd = df_clean[df_clean['target'] == 1]
df_normal = df_clean[df_clean['target'] == 0]

pd_vals = df_pd[feature].values
normal_vals = df_normal[feature].values

stats_dict = {
    'Metric': ['Count', 'Mean', 'Std', 'Min', '1%', '5%', '10%', '25%', '50%', '75%', '90%', '95%', '99%', 'Max'],
    'PD': [
        len(pd_vals),
        np.mean(pd_vals),
        np.std(pd_vals),
        np.min(pd_vals),
        np.percentile(pd_vals, 1),
        np.percentile(pd_vals, 5),
        np.percentile(pd_vals, 10),
        np.percentile(pd_vals, 25),
        np.percentile(pd_vals, 50),
        np.percentile(pd_vals, 75),
        np.percentile(pd_vals, 90),
        np.percentile(pd_vals, 95),
        np.percentile(pd_vals, 99),
        np.max(pd_vals)
    ],
    'Normal': [
        len(normal_vals),
        np.mean(normal_vals),
        np.std(normal_vals),
        np.min(normal_vals),
        np.percentile(normal_vals, 1),
        np.percentile(normal_vals, 5),
        np.percentile(normal_vals, 10),
        np.percentile(normal_vals, 25),
        np.percentile(normal_vals, 50),
        np.percentile(normal_vals, 75),
        np.percentile(normal_vals, 90),
        np.percentile(normal_vals, 95),
        np.percentile(normal_vals, 99),
        np.max(normal_vals)
    ]
}

df_stats = pd.DataFrame(stats_dict)
df_stats['Difference'] = df_stats['PD'] - df_stats['Normal']
print("\n", df_stats.to_string(index=False))

# ==================== OUTLIER DETECTION ====================
print("\n" + "="*80)
print("🎯 OUTLIER DETECTION")
print("="*80)

def detect_outliers_iqr(data, multiplier=1.5):
    """Detect outliers using IQR method"""
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - multiplier * iqr
    upper_bound = q3 + multiplier * iqr
    outliers = (data < lower_bound) | (data > upper_bound)
    return outliers, lower_bound, upper_bound

# PD outliers
pd_outliers, pd_lower, pd_upper = detect_outliers_iqr(pd_vals)
print(f"\nPD Group:")
print(f"  IQR bounds: [{pd_lower:.2f}, {pd_upper:.2f}]")
print(f"  Outliers: {pd_outliers.sum()} ({pd_outliers.sum()/len(pd_vals)*100:.1f}%)")
if pd_outliers.sum() > 0:
    print(f"  Outlier values: {pd_vals[pd_outliers]}")

# Normal outliers
normal_outliers, normal_lower, normal_upper = detect_outliers_iqr(normal_vals)
print(f"\nNormal Group:")
print(f"  IQR bounds: [{normal_lower:.2f}, {normal_upper:.2f}]")
print(f"  Outliers: {normal_outliers.sum()} ({normal_outliers.sum()/len(normal_vals)*100:.1f}%)")
if normal_outliers.sum() > 5:
    print(f"  Outlier range: [{np.min(normal_vals[normal_outliers]):.2f}, {np.max(normal_vals[normal_outliers]):.2f}]")

# ==================== VALUE DISTRIBUTION ANALYSIS ====================
print("\n" + "="*80)
print("🔬 VALUE DISTRIBUTION ANALYSIS")
print("="*80)

# Check for specific value patterns
print("\nUnique values analysis:")
print(f"  PD unique values: {len(np.unique(pd_vals))}")
print(f"  Normal unique values: {len(np.unique(normal_vals))}")

# Check for common values
all_vals = np.concatenate([pd_vals, normal_vals])
value_counts_all = pd.Series(all_vals).value_counts()
print(f"\nMost common values (overall):")
print(value_counts_all.head(10))

# Check for values that are exclusive or heavily skewed to one group
print("\n" + "="*80)
print("🎯 DISCRIMINATIVE VALUE ANALYSIS")
print("="*80)

# Create bins for analysis
bins = np.percentile(all_vals, np.linspace(0, 100, 21))  # 20 bins
pd_hist, _ = np.histogram(pd_vals, bins=bins)
normal_hist, _ = np.histogram(normal_vals, bins=bins)

# Find bins with extreme ratios
ratios = []
for i in range(len(pd_hist)):
    if normal_hist[i] > 0 and pd_hist[i] > 0:
        ratio = pd_hist[i] / normal_hist[i]
        ratios.append({
            'Bin': f'[{bins[i]:.1f}, {bins[i+1]:.1f}]',
            'PD_count': pd_hist[i],
            'Normal_count': normal_hist[i],
            'PD_pct': pd_hist[i] / len(pd_vals) * 100,
            'Normal_pct': normal_hist[i] / len(normal_vals) * 100,
            'Ratio_PD_to_Normal': ratio
        })
    elif pd_hist[i] > 0:
        ratios.append({
            'Bin': f'[{bins[i]:.1f}, {bins[i+1]:.1f}]',
            'PD_count': pd_hist[i],
            'Normal_count': 0,
            'PD_pct': pd_hist[i] / len(pd_vals) * 100,
            'Normal_pct': 0,
            'Ratio_PD_to_Normal': np.inf
        })
    elif normal_hist[i] > 0:
        ratios.append({
            'Bin': f'[{bins[i]:.1f}, {bins[i+1]:.1f}]',
            'PD_count': 0,
            'Normal_count': normal_hist[i],
            'PD_pct': 0,
            'Normal_pct': normal_hist[i] / len(normal_vals) * 100,
            'Ratio_PD_to_Normal': 0
        })

df_ratios = pd.DataFrame(ratios)
df_ratios_sorted = df_ratios.sort_values('Ratio_PD_to_Normal', ascending=False)

print("\nBins with highest PD concentration (top 10):")
print(df_ratios_sorted.head(10).to_string(index=False))

print("\nBins with highest Normal concentration (bottom 10):")
print(df_ratios_sorted.tail(10).to_string(index=False))

# ==================== VISUALIZATIONS ====================
print("\n" + "="*80)
print("🎨 CREATING VISUALIZATIONS")
print("="*80)

# Figure 1: Detailed Distribution Comparison
print("\n📊 Creating detailed distribution plots...")
fig = plt.figure(figsize=(20, 12))

# Subplot 1: Histograms with KDE
ax1 = plt.subplot(2, 3, 1)
ax1.hist(normal_vals, bins=50, alpha=0.5, color='#4ECDC4', label='Normal', density=True, edgecolor='black')
ax1.hist(pd_vals, bins=30, alpha=0.5, color='#FF6B6B', label='Panic Disorder', density=True, edgecolor='black')

from scipy.stats import gaussian_kde
kde_normal = gaussian_kde(normal_vals)
kde_pd = gaussian_kde(pd_vals)
x_range = np.linspace(min(all_vals), max(all_vals), 1000)
ax1.plot(x_range, kde_normal(x_range), color='#4ECDC4', linewidth=2.5, label='Normal KDE')
ax1.plot(x_range, kde_pd(x_range), color='#FF6B6B', linewidth=2.5, label='PD KDE')

ax1.set_xlabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax1.set_ylabel('Density', fontsize=12)
ax1.set_title('Distribution Comparison with KDE', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Subplot 2: Box plots with outliers
ax2 = plt.subplot(2, 3, 2)
bp_data = [normal_vals, pd_vals]
bp = ax2.boxplot(bp_data, labels=['Normal', 'PD'], patch_artist=True, showfliers=True)
bp['boxes'][0].set_facecolor('#4ECDC4')
bp['boxes'][1].set_facecolor('#FF6B6B')
ax2.set_ylabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax2.set_title('Box Plot Comparison\n(showing outliers)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Subplot 3: Violin plots
ax3 = plt.subplot(2, 3, 3)
parts = ax3.violinplot([normal_vals, pd_vals], positions=[0, 1], showmeans=True, showmedians=True)
colors = ['#4ECDC4', '#FF6B6B']
for pc, color in zip(parts['bodies'], colors):
    pc.set_facecolor(color)
    pc.set_alpha(0.7)
ax3.set_xticks([0, 1])
ax3.set_xticklabels(['Normal', 'PD'])
ax3.set_ylabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax3.set_title('Violin Plot Comparison', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')

# Subplot 4: Cumulative Distribution Functions
ax4 = plt.subplot(2, 3, 4)
normal_sorted = np.sort(normal_vals)
pd_sorted = np.sort(pd_vals)
normal_cdf = np.arange(1, len(normal_sorted) + 1) / len(normal_sorted)
pd_cdf = np.arange(1, len(pd_sorted) + 1) / len(pd_sorted)

ax4.plot(normal_sorted, normal_cdf, color='#4ECDC4', linewidth=2, label='Normal')
ax4.plot(pd_sorted, pd_cdf, color='#FF6B6B', linewidth=2, label='PD')
ax4.set_xlabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax4.set_ylabel('Cumulative Probability', fontsize=12)
ax4.set_title('Cumulative Distribution Functions', fontsize=14, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Calculate max distance between CDFs (Kolmogorov-Smirnov statistic)
ks_stat, ks_pval = stats.ks_2samp(pd_vals, normal_vals)
ax4.text(0.05, 0.95, f'KS statistic: {ks_stat:.3f}\np-value: {ks_pval:.2e}',
         transform=ax4.transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Subplot 5: Q-Q Plot
ax5 = plt.subplot(2, 3, 5)
stats.probplot(pd_vals, dist="norm", plot=ax5)
ax5.set_title('Q-Q Plot: PD Group vs Normal Distribution', fontsize=14, fontweight='bold')
ax5.grid(True, alpha=0.3)

# Subplot 6: Scatter plot with jitter
ax6 = plt.subplot(2, 3, 6)
np.random.seed(42)
jitter_normal = np.random.normal(0, 0.04, len(normal_vals))
jitter_pd = np.random.normal(1, 0.04, len(pd_vals))

# Sample if too many points
max_points = 500
if len(normal_vals) > max_points:
    sample_idx = np.random.choice(len(normal_vals), max_points, replace=False)
    ax6.scatter(jitter_normal[sample_idx], normal_vals[sample_idx], 
               alpha=0.3, s=20, color='#4ECDC4', edgecolors='black', linewidths=0.5, label='Normal')
else:
    ax6.scatter(jitter_normal, normal_vals, 
               alpha=0.3, s=20, color='#4ECDC4', edgecolors='black', linewidths=0.5, label='Normal')

ax6.scatter(jitter_pd, pd_vals, 
           alpha=0.5, s=30, color='#FF6B6B', edgecolors='black', linewidths=0.5, label='PD')

ax6.set_xticks([0, 1])
ax6.set_xticklabels(['Normal', 'PD'])
ax6.set_ylabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax6.set_title('Individual Values (with jitter)', fontsize=14, fontweight='bold')
ax6.legend()
ax6.grid(True, alpha=0.3, axis='y')

plt.suptitle('WHQ_WHD050: Deep Distribution Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure1_comprehensive_distribution_analysis.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Saved: {output_fig}")
plt.close()

# Figure 2: Bin-wise Discrimination
print("\n📊 Creating bin-wise discrimination plot...")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Top plot: Stacked histogram
bin_edges = np.percentile(all_vals, np.linspace(0, 100, 31))
ax1.hist([normal_vals, pd_vals], bins=bin_edges, stacked=False, 
         label=['Normal', 'PD'], color=['#4ECDC4', '#FF6B6B'], 
         alpha=0.7, edgecolor='black')
ax1.set_xlabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_title('Histogram by Group (30 bins)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Bottom plot: Ratio by bin
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
pd_hist_plot, _ = np.histogram(pd_vals, bins=bin_edges)
normal_hist_plot, _ = np.histogram(normal_vals, bins=bin_edges)

# Calculate ratio (with smoothing to avoid division by zero)
ratio_plot = np.where(normal_hist_plot > 0, 
                      pd_hist_plot / normal_hist_plot, 
                      0)

colors_ratio = ['red' if r > 1 else 'green' for r in ratio_plot]
ax2.bar(bin_centers, ratio_plot, width=np.diff(bin_edges), 
        alpha=0.7, edgecolor='black', color=colors_ratio)
ax2.axhline(y=1, color='black', linestyle='--', linewidth=2, label='Equal distribution')
ax2.set_xlabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax2.set_ylabel('PD / Normal Ratio', fontsize=12)
ax2.set_title('Discrimination Power by Value Range\n(Red = more PD, Green = more Normal)', 
              fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_yscale('log')

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure2_binwise_discrimination.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Saved: {output_fig}")
plt.close()

# Figure 3: Percentile Analysis
print("\n📊 Creating percentile comparison...")
fig, ax = plt.subplots(figsize=(14, 8))

percentiles = np.arange(0, 101, 5)
pd_percentiles = [np.percentile(pd_vals, p) for p in percentiles]
normal_percentiles = [np.percentile(normal_vals, p) for p in percentiles]

ax.plot(percentiles, pd_percentiles, marker='o', markersize=8, 
        linewidth=2.5, color='#FF6B6B', label='PD')
ax.plot(percentiles, normal_percentiles, marker='s', markersize=8, 
        linewidth=2.5, color='#4ECDC4', label='Normal')

# Fill between to show difference
ax.fill_between(percentiles, pd_percentiles, normal_percentiles, 
                alpha=0.2, color='gray', label='Difference')

ax.set_xlabel('Percentile', fontsize=12)
ax.set_ylabel('Weight 1 Year Ago (lbs)', fontsize=12)
ax.set_title('Percentile Comparison: PD vs Normal', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure3_percentile_comparison.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"✅ Saved: {output_fig}")
plt.close()

# ==================== SEPARATION MECHANISM ANALYSIS ====================
print("\n" + "="*80)
print("🔬 ANALYZING SEPARATION MECHANISM")
print("="*80)

# Calculate overlap in different ways
print("\n1. Simple Range Overlap:")
pd_min, pd_max = np.min(pd_vals), np.max(pd_vals)
normal_min, normal_max = np.min(normal_vals), np.max(normal_vals)
print(f"   PD range: [{pd_min:.2f}, {pd_max:.2f}]")
print(f"   Normal range: [{normal_min:.2f}, {normal_max:.2f}]")

overlap_min = max(pd_min, normal_min)
overlap_max = min(pd_max, normal_max)
if overlap_max > overlap_min:
    overlap_range = overlap_max - overlap_min
    pd_range = pd_max - pd_min
    normal_range = normal_max - normal_min
    print(f"   Overlap range: [{overlap_min:.2f}, {overlap_max:.2f}]")
    print(f"   Overlap as % of PD range: {overlap_range/pd_range*100:.1f}%")
    print(f"   Overlap as % of Normal range: {overlap_range/normal_range*100:.1f}%")
else:
    print("   ⚠️  NO OVERLAP IN RANGES!")

# Check if there's a perfect cutoff
print("\n2. Perfect Cutoff Analysis:")
all_vals_sorted = np.sort(np.unique(all_vals))
best_cutoff = None
best_accuracy = 0

for cutoff in all_vals_sorted:
    # Try cutoff (PD below, Normal above)
    pd_correct = np.sum(pd_vals <= cutoff)
    normal_correct = np.sum(normal_vals > cutoff)
    accuracy1 = (pd_correct + normal_correct) / (len(pd_vals) + len(normal_vals))
    
    # Try inverse (PD above, Normal below)
    pd_correct_inv = np.sum(pd_vals > cutoff)
    normal_correct_inv = np.sum(normal_vals <= cutoff)
    accuracy2 = (pd_correct_inv + normal_correct_inv) / (len(pd_vals) + len(normal_vals))
    
    if accuracy1 > best_accuracy:
        best_accuracy = accuracy1
        best_cutoff = (cutoff, 'PD<=cutoff, Normal>cutoff')
    
    if accuracy2 > best_accuracy:
        best_accuracy = accuracy2
        best_cutoff = (cutoff, 'PD>cutoff, Normal<=cutoff')

if best_cutoff:
    print(f"   Best single cutoff: {best_cutoff[0]:.2f}")
    print(f"   Rule: {best_cutoff[1]}")
    print(f"   Accuracy: {best_accuracy*100:.2f}%")
    
    if best_accuracy > 0.95:
        print("   🎯 FOUND NEAR-PERFECT DISCRIMINATOR!")

# ==================== SAVE SUMMARY REPORT ====================
print("\n" + "="*80)
print("📝 GENERATING INVESTIGATION REPORT")
print("="*80)

report = f"""
WHQ_WHD050 INVESTIGATION REPORT
================================

Feature: Weight 1 Year Ago (WHQ_WHD050)
Investigation Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

EXECUTIVE SUMMARY
-----------------
This feature showed 98.5% separation between PD and Normal groups in Phase 1 analysis,
despite having only a moderate Cohen's d. This investigation reveals the mechanism.

MISSING DATA ANALYSIS
---------------------
Total missing: {missing_full} ({missing_pct:.2f}%)
PD missing: {missing_pd}/{total_pd} ({missing_pd/total_pd*100:.2f}%)
Normal missing: {missing_normal}/{total_normal} ({missing_normal/total_normal*100:.2f}%)

DESCRIPTIVE STATISTICS
----------------------
                    PD              Normal          Difference
Mean:            {np.mean(pd_vals):8.2f}         {np.mean(normal_vals):8.2f}         {np.mean(pd_vals)-np.mean(normal_vals):8.2f}
Median:          {np.median(pd_vals):8.2f}         {np.median(normal_vals):8.2f}         {np.median(pd_vals)-np.median(normal_vals):8.2f}
Std:             {np.std(pd_vals):8.2f}         {np.std(normal_vals):8.2f}         {np.std(pd_vals)-np.std(normal_vals):8.2f}
Range:           [{pd_min:6.2f}, {pd_max:6.2f}]  [{normal_min:6.2f}, {normal_max:6.2f}]

OUTLIER ANALYSIS
----------------
PD outliers (IQR method): {pd_outliers.sum()} ({pd_outliers.sum()/len(pd_vals)*100:.1f}%)
Normal outliers (IQR method): {normal_outliers.sum()} ({normal_outliers.sum()/len(normal_vals)*100:.1f}%)

BEST SINGLE CUTOFF
------------------
Value: {best_cutoff[0]:.2f} lbs
Rule: {best_cutoff[1]}
Accuracy: {best_accuracy*100:.2f}%

STATISTICAL TESTS
-----------------
Kolmogorov-Smirnov test: D = {ks_stat:.3f}, p < {ks_pval:.2e}
{"HIGHLY SIGNIFICANT DIFFERENCE IN DISTRIBUTIONS" if ks_pval < 0.001 else ""}

KEY FINDINGS
------------
1. {"CRITICAL: Near-perfect single-variable discrimination found!" if best_accuracy > 0.95 else "Moderate single-variable discrimination"}
2. {"Substantial missingness in data - may affect generalizability" if missing_pct > 30 else "Low missingness - data quality good"}
3. {"Significant outliers detected - may drive separation" if (pd_outliers.sum() + normal_outliers.sum()) > 50 else "Outliers minimal"}
4. Distribution shapes: {"Different" if ks_pval < 0.001 else "Similar"}

INTERPRETATION
--------------
The 98.5% separation observed in Phase 1 analysis appears to be {"driven by a clear single cutoff threshold" if best_accuracy > 0.90 else "due to distributional differences rather than a single threshold"}.

This {'suggests a strong univariate discriminator' if best_accuracy > 0.90 else 'indicates complex multivariate interactions'}.

RECOMMENDATION
--------------
{'⚠️  This feature alone provides near-perfect classification. Investigate if it should be excluded or if it represents true clinical signal.' if best_accuracy > 0.95 else '✓ This feature contributes meaningfully but not dominantly to the model.'}

Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

output_report = OUTPUT_DIR / 'WHD050_INVESTIGATION_REPORT.txt'
with open(output_report, 'w') as f:
    f.write(report)
print(f"\n✅ Report saved: {output_report}")

# Save detailed statistics
df_stats.to_csv(OUTPUT_DIR / 'detailed_statistics.csv', index=False)
df_ratios.to_csv(OUTPUT_DIR / 'binwise_ratios.csv', index=False)

print("\n" + "="*80)
print("✅ INVESTIGATION COMPLETE!")
print("="*80)
print(f"\n📂 All results saved to: {OUTPUT_DIR}")
print("\n📊 Generated files:")
print("   - WHD050_INVESTIGATION_REPORT.txt")
print("   - Figure1_comprehensive_distribution_analysis.png")
print("   - Figure2_binwise_discrimination.png")
print("   - Figure3_percentile_comparison.png")
print("   - detailed_statistics.csv")
print("   - binwise_ratios.csv")

# Print summary to console
print("\n" + "="*80)
print("🎯 QUICK SUMMARY")
print("="*80)
print(f"\nBest single cutoff accuracy: {best_accuracy*100:.2f}%")
if best_accuracy > 0.95:
    print("⚠️  WARNING: This single feature can nearly perfectly discriminate!")
    print("   This is HIGHLY UNUSUAL and may indicate:")
    print("   1. A true strong clinical marker")
    print("   2. Data leakage or collection artifact")
    print("   3. Need for external validation")
print(f"\nMissingness: {missing_pct:.1f}% (may affect results)")
print(f"Outliers: PD={pd_outliers.sum()}, Normal={normal_outliers.sum()}")

🔍 DEEP INVESTIGATION: WHQ_WHD050 (Weight 1 Year Ago)

📊 Loading datasets...
   Full dataset: 6581 samples
   Clean dataset: 3279 samples
   Dropped: 3302 samples (50.2%)

🔍 MISSING VALUE ANALYSIS

Total missing values: 4 (0.06%)

Missing in PD group: 0/178 (0.00%)
Missing in Normal group: 4/6403 (0.06%)
⚠️  WARNING: Much higher missingness in Normal group!

📊 BASIC STATISTICS (Clean Data)

 Metric         PD       Normal    Difference
 Count 115.000000  3164.000000  -3049.000000
  Mean 152.695652   574.006321   -421.310669
   Std  33.212144  6154.668204  -6121.456060
   Min 110.000000    68.000000     42.000000
    1% 110.000000    98.000000     12.000000
    5% 110.000000   110.000000      0.000000
   10% 118.000000   120.000000     -2.000000
   25% 120.000000   140.000000    -20.000000
   50% 139.000000   165.000000    -26.000000
   75% 166.500000   199.000000    -32.500000
   90% 200.000000   235.000000    -35.000000
   95% 225.000000   250.000000    -25.000000
   99% 225.000000   3

In [6]:
"""
DATASET CLEANUP: Remove Contaminated Feature WHQ_WHD050
========================================================
WHQ_WHD050 (Weight 1 Year Ago) contains 99999 coded values that create
artificial separation (96.49% accuracy). We remove this feature to work
with clean data only.

Author: Panic Disorder ML Investigation
Date: 2025-11-11
"""

import pandas as pd
import numpy as np
from pathlib import Path

print("="*80)
print("🧹 DATASET CLEANUP: Removing WHQ_WHD050")
print("="*80)

# ==================== LOAD ORIGINAL DATA ====================
print("\n📊 Loading original dataset...")
DATA_PATH = '/Users/filipecarvalho/Documents/data_science_projects/Panic.3/NHANES_panic_12features.csv'
df_original = pd.read_csv(DATA_PATH)
print(f"   ✅ Loaded: {df_original.shape[0]} samples × {df_original.shape[1]} columns")

# ==================== DEFINE 11 CLEAN FEATURES ====================
print("\n" + "="*80)
print("📋 DEFINING 11 CLEAN FEATURES")
print("="*80)

# Original 12 features (ordered by importance from Paper 2)
features_original = [
    'BIX_BIDFAT',      # 1. Body Fat Mass (23.8%)
    'DEMO_INDFMPIR',   # 2. Poverty Income Ratio (17.4%)
    'DEMO_RIDAGEMN',   # 3. Age in months (10.9%)
    'DEMO_RIAGENDR',   # 4. Gender (10.6%)
    'BPX_BPXDAR',      # 5. Diastolic BP (9.8%)
    'WHQ_WHD050',      # 6. Weight 1 Year Ago (8.0%) ⚠️ CONTAMINATED - REMOVING
    'DEMO_DMDHHSIZ',   # 7. Household Size (7.9%)
    'BMX_BMXBMI',      # 8. BMI (7.4%)
    'ALQ_ALQ130',      # 9. Alcohol Consumption (2.6%)
    'MCQ_MCQ250F',     # 10. Family History HTN/Stroke (0.9%)
    'MPQ_MPQ070',      # 11. Lower Back Pain (0.5%)
    'DEMO_DMDMARTL'    # 12. Marital Status (0.3%)
]

# New 11 features (removing WHQ_WHD050)
features_clean = [
    'BIX_BIDFAT',      # 1. Body Fat Mass
    'DEMO_INDFMPIR',   # 2. Poverty Income Ratio
    'DEMO_RIDAGEMN',   # 3. Age in months
    'DEMO_RIAGENDR',   # 4. Gender
    'BPX_BPXDAR',      # 5. Diastolic BP
    'DEMO_DMDHHSIZ',   # 6. Household Size
    'BMX_BMXBMI',      # 7. BMI
    'ALQ_ALQ130',      # 8. Alcohol Consumption
    'MCQ_MCQ250F',     # 9. Family History HTN/Stroke
    'MPQ_MPQ070',      # 10. Lower Back Pain
    'DEMO_DMDMARTL'    # 11. Marital Status
]

print("\n✅ 11 Clean Features Selected:")
for i, feat in enumerate(features_clean, 1):
    print(f"   {i:2d}. {feat}")

print("\n❌ Removed Feature:")
print("    WHQ_WHD050 (Weight 1 Year Ago) - Contains 99999 coded values")

# ==================== CREATE CLEAN DATASET ====================
print("\n" + "="*80)
print("📦 CREATING CLEAN DATASET")
print("="*80)

# Select only the 11 clean features + target
df_clean = df_original[features_clean + ['target']].copy()

print(f"\n✅ New dataset shape: {df_clean.shape[0]} samples × {df_clean.shape[1]} columns")
print(f"   Features: {len(features_clean)}")
print(f"   Target: 1")

# ==================== DATA QUALITY CHECK ====================
print("\n" + "="*80)
print("🔍 DATA QUALITY CHECK")
print("="*80)

# Check for missing values
print("\n1. Missing Values:")
missing_counts = df_clean.isnull().sum()
total_missing = missing_counts.sum()
print(f"   Total missing values: {total_missing}")

if total_missing > 0:
    print("\n   Missing values by feature:")
    for feat in features_clean:
        missing = missing_counts[feat]
        if missing > 0:
            pct = (missing / len(df_clean)) * 100
            print(f"      {feat:20s}: {missing:4d} ({pct:5.2f}%)")
else:
    print("   ✅ No missing values in any feature!")

# Check for suspicious values (99999, 9999, etc)
print("\n2. Checking for Suspicious Coded Values (99999, 9999, 999):")
suspicious_found = False

for feat in features_clean:
    # Check for extreme values that might be codes
    feat_values = df_clean[feat].values
    feat_values_no_nan = feat_values[~np.isnan(feat_values)]
    
    if len(feat_values_no_nan) > 0:
        max_val = np.max(feat_values_no_nan)
        
        # Check for common NHANES missing codes
        suspicious_codes = [99999, 9999, 999, 99, 77, 66]
        found_codes = []
        
        for code in suspicious_codes:
            count = np.sum(feat_values_no_nan == code)
            if count > 0:
                found_codes.append((code, count))
        
        if found_codes:
            suspicious_found = True
            print(f"   ⚠️  {feat}:")
            for code, count in found_codes:
                print(f"      Value {code}: {count} occurrences ({count/len(feat_values_no_nan)*100:.2f}%)")

if not suspicious_found:
    print("   ✅ No suspicious coded values detected!")

# Check target distribution
print("\n3. Target Distribution:")
target_counts = df_clean['target'].value_counts().sort_index()
for val, count in target_counts.items():
    label = "Normal" if val == 0 else "Panic Disorder"
    pct = (count / len(df_clean)) * 100
    print(f"   {label:15s} (target={val}): {count:5d} ({pct:5.2f}%)")

# Basic statistics
print("\n4. Basic Statistics for 11 Features:")
print(df_clean[features_clean].describe().round(2))

# ==================== SAVE CLEAN DATASET ====================
print("\n" + "="*80)
print("💾 SAVING CLEAN DATASET")
print("="*80)

# Save with descriptive name
output_path = '/Users/filipecarvalho/Documents/data_science_projects/Panic.3/NHANES_panic_11features_CLEAN.csv'
df_clean.to_csv(output_path, index=False)
print(f"\n✅ Clean dataset saved to:")
print(f"   {output_path}")

# Also save a summary file
summary_path = Path('/Users/filipecarvalho/Documents/data_science_projects/Panic.3/results')
summary_path.mkdir(exist_ok=True)

summary_file = summary_path / 'DATASET_CLEANUP_SUMMARY.txt'
summary_text = f"""
DATASET CLEANUP SUMMARY
========================

Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

CHANGES MADE
------------
Original dataset: NHANES_panic_12features.csv
New dataset: NHANES_panic_11features_CLEAN.csv

REMOVED FEATURE
---------------
WHQ_WHD050 (Weight 1 Year Ago)

Reason: Contains 99999 coded values (NHANES "Don't Know/Refused" code)
        that created artificial 96.49% discrimination between groups.
        This is a data quality artifact, not a true clinical signal.

11 CLEAN FEATURES RETAINED
---------------------------
1. BIX_BIDFAT      - Body Fat Mass (kg)
2. DEMO_INDFMPIR   - Poverty Income Ratio
3. DEMO_RIDAGEMN   - Age (months)
4. DEMO_RIAGENDR   - Gender
5. BPX_BPXDAR      - Diastolic Blood Pressure
6. DEMO_DMDHHSIZ   - Household Size
7. BMX_BMXBMI      - Body Mass Index
8. ALQ_ALQ130      - Alcohol Consumption (drinks/day)
9. MCQ_MCQ250F     - Family History HTN/Stroke
10. MPQ_MPQ070     - Lower Back Pain (last 3 months)
11. DEMO_DMDMARTL  - Marital Status

DATASET STATISTICS
------------------
Total samples: {len(df_clean)}
Features: 11
Target variable: 1 (CIDPSCOR)

Target Distribution:
- Panic Disorder: {target_counts[1]} ({target_counts[1]/len(df_clean)*100:.2f}%)
- Normal: {target_counts[0]} ({target_counts[0]/len(df_clean)*100:.2f}%)

Missing Values: {total_missing} ({total_missing/(len(df_clean)*len(features_clean))*100:.3f}% of all values)

NEXT STEPS
----------
1. Re-train the machine learning model with 11 clean features
2. Verify if 100% accuracy is maintained with clean data
3. Re-run Phase 1.1 distributional analysis with 11 features
4. Compare new results with original 12-feature results
5. Update paper methodology to document this data cleaning step

IMPACT ASSESSMENT
-----------------
The removal of WHQ_WHD050 is expected to:
- Reduce artificial discrimination power
- Provide more realistic model performance estimates
- Improve model generalizability
- Ensure clinical validity of findings

If the model still achieves near-perfect accuracy with 11 clean features,
this strengthens the validity of the findings. If accuracy drops significantly,
this indicates the original results were partly driven by data artifacts.

Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

with open(summary_file, 'w') as f:
    f.write(summary_text)
print(f"\n✅ Summary saved to:")
print(f"   {summary_file}")

# ==================== COMPARISON WITH ORIGINAL ====================
print("\n" + "="*80)
print("📊 COMPARISON WITH ORIGINAL DATASET")
print("="*80)

print(f"\nOriginal (12 features): {df_original.shape[0]} × {df_original.shape[1]}")
print(f"Clean (11 features):    {df_clean.shape[0]} × {df_clean.shape[1]}")
print(f"Reduction: {df_original.shape[1] - df_clean.shape[1]} column removed")

# ==================== FINAL INSTRUCTIONS ====================
print("\n" + "="*80)
print("✅ CLEANUP COMPLETE!")
print("="*80)

print("\n📋 NEXT STEPS:")
print("\n1. 🤖 RE-TRAIN MODEL:")
print("   - Load NHANES_panic_11features_CLEAN.csv")
print("   - Train same model architecture (XGBoost, Random Forest, etc.)")
print("   - Compare accuracy with original 12-feature model")
print("   - Save new model as 'panic_model_11features_CLEAN.joblib'")

print("\n2. 📊 RE-RUN PHASE 1.1 ANALYSIS:")
print("   - Use the 11-feature dataset")
print("   - Generate new distribution plots")
print("   - Compare effect sizes without WHQ_WHD050")

print("\n3. 🔍 VERIFY MODEL BEHAVIOR:")
print("   - Check if 100% accuracy is maintained")
print("   - Analyze feature importances in new model")
print("   - Verify no other features have suspicious patterns")

print("\n4. 📝 UPDATE PAPER:")
print("   - Document data cleaning process")
print("   - Explain removal of WHQ_WHD050")
print("   - Present results from clean 11-feature model")
print("   - Discuss implications for findings")

print("\n🎯 KEY QUESTION:")
print("   Does the model still achieve near-perfect separation with 11 clean features?")
print("   This will determine if the phenomenon is real or artifact-driven!")

print("\n" + "="*80)

🧹 DATASET CLEANUP: Removing WHQ_WHD050

📊 Loading original dataset...
   ✅ Loaded: 6581 samples × 13 columns

📋 DEFINING 11 CLEAN FEATURES

✅ 11 Clean Features Selected:
    1. BIX_BIDFAT
    2. DEMO_INDFMPIR
    3. DEMO_RIDAGEMN
    4. DEMO_RIAGENDR
    5. BPX_BPXDAR
    6. DEMO_DMDHHSIZ
    7. BMX_BMXBMI
    8. ALQ_ALQ130
    9. MCQ_MCQ250F
   10. MPQ_MPQ070
   11. DEMO_DMDMARTL

❌ Removed Feature:
    WHQ_WHD050 (Weight 1 Year Ago) - Contains 99999 coded values

📦 CREATING CLEAN DATASET

✅ New dataset shape: 6581 samples × 12 columns
   Features: 11
   Target: 1

🔍 DATA QUALITY CHECK

1. Missing Values:
   Total missing values: 5079

   Missing values by feature:
      BIX_BIDFAT          : 1901 (28.89%)
      DEMO_INDFMPIR       :  423 ( 6.43%)
      BPX_BPXDAR          :  429 ( 6.52%)
      BMX_BMXBMI          :  233 ( 3.54%)
      ALQ_ALQ130          : 1917 (29.13%)
      DEMO_DMDMARTL       :  176 ( 2.67%)

2. Checking for Suspicious Coded Values (99999, 9999, 999):
   ⚠️  BPX_B

In [1]:
"""
RE-TRAINING MODEL WITH 11 CLEAN FEATURES
=========================================
Testing if 100% accuracy is maintained after removing WHQ_WHD050

Uses EXACT same pipeline, hyperparameters, and preprocessing as original model
to ensure fair comparison.

Original: 12 features (including contaminated WHQ_WHD050)
New: 11 clean features (WHQ_WHD050 removed)

Author: Panic Disorder ML Investigation
Date: 2025-11-11
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import (
    classification_report, roc_auc_score, roc_curve, confusion_matrix,
    accuracy_score, precision_score, recall_score, f1_score
)
import joblib
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

print("="*80)
print("🤖 RE-TRAINING MODEL WITH 11 CLEAN FEATURES")
print("="*80)

# Paths
DATA_PATH = '/Users/filipecarvalho/Documents/data_science_projects/Panic.3/NHANES_panic_11features_CLEAN.csv'
OUTPUT_DIR = Path('/Users/filipecarvalho/Documents/data_science_projects/Panic.3/results/model_retrain_11features')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 11 Clean Features (EXACT same order as original, minus WHQ_WHD050)
FEATURES_CLEAN = [
    'BPX_BPXDAR',      # Diastolic BP
    'MPQ_MPQ070',      # Lower Back Pain
    'DEMO_RIDAGEMN',   # Age (months)
    'DEMO_INDFMPIR',   # Poverty Income Ratio
    'ALQ_ALQ130',      # Alcohol Consumption
    'BMX_BMXBMI',      # BMI
    'DEMO_DMDMARTL',   # Marital Status
    'DEMO_RIAGENDR',   # Gender
    'BIX_BIDFAT',      # Body Fat Mass
    'MCQ_MCQ250F',     # Family History HTN/Stroke
    'DEMO_DMDHHSIZ'    # Household Size
]

# EXACT hyperparameters from original model
MODEL_PARAMS = {
    'n_estimators': 200,
    'max_depth': 8,
    'learning_rate': 0.1,
    'min_samples_leaf': 1,
    'min_samples_split': 4,
    'random_state': 42
}

RANDOM_STATE = 42
TEST_SIZE = 0.25

print(f"\n📂 Data: {DATA_PATH}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"\n📋 Using {len(FEATURES_CLEAN)} clean features")
print(f"🎲 Random state: {RANDOM_STATE}")
print(f"📊 Test size: {TEST_SIZE*100}%")

# ============================================================================
# 1. LOAD DATA
# ============================================================================

print("\n" + "="*80)
print("1. LOADING CLEAN DATASET")
print("="*80)

df = pd.read_csv(DATA_PATH)
print(f"\n✅ Loaded: {df.shape[0]} samples × {df.shape[1]} columns")

# Verify all features are present
missing_features = [f for f in FEATURES_CLEAN if f not in df.columns]
if missing_features:
    print(f"\n❌ ERROR: Missing features: {missing_features}")
    exit(1)
else:
    print(f"✅ All {len(FEATURES_CLEAN)} features present")

# Check target
if 'target' not in df.columns:
    print("\n❌ ERROR: 'target' column not found!")
    exit(1)

print(f"\n🎯 Target distribution:")
target_counts = df['target'].value_counts().sort_index()
for val, count in target_counts.items():
    label = "Normal" if val == 0 else "Panic Disorder"
    pct = (count / len(df)) * 100
    print(f"   {label:15s} (target={val}): {count:5d} ({pct:5.2f}%)")

prevalence = df['target'].mean()
print(f"\n📊 PD Prevalence: {prevalence*100:.2f}%")

# ============================================================================
# 2. PREPROCESSING (EXACT SAME AS ORIGINAL)
# ============================================================================

print("\n" + "="*80)
print("2. PREPROCESSING")
print("="*80)

print("\n🔧 Imputing missing values with median (same as original)...")

# Create a copy for processing
df_processed = df.copy()

# Get numeric columns (features only, not target)
numeric_cols = [col for col in FEATURES_CLEAN if df[col].dtype in ['int64', 'float64']]

# Impute missing values with median (EXACT same as original)
for col in numeric_cols:
    if df_processed[col].isnull().sum() > 0:
        imputer = SimpleImputer(strategy='median')
        df_processed[col] = imputer.fit_transform(df_processed[[col]])
        print(f"   Imputed {col}")

# Check for any remaining missing values
missing_after = df_processed[FEATURES_CLEAN].isnull().sum().sum()
if missing_after > 0:
    print(f"\n⚠️  Warning: {missing_after} missing values remaining after imputation")
    print("   Dropping rows with missing values...")
    df_processed = df_processed.dropna(subset=FEATURES_CLEAN)
    print(f"   New shape: {df_processed.shape}")
else:
    print(f"\n✅ No missing values after imputation")

# Extract features and target
X = df_processed[FEATURES_CLEAN]
y = df_processed['target']

print(f"\n✅ Final dataset:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"   Features: {len(FEATURES_CLEAN)}")

# ============================================================================
# 3. TRAIN/TEST SPLIT (EXACT SAME AS ORIGINAL)
# ============================================================================

print("\n" + "="*80)
print("3. TRAIN/TEST SPLIT")
print("="*80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    stratify=y, 
    random_state=RANDOM_STATE
)

print(f"\n📊 Split summary:")
print(f"   Train: {X_train.shape[0]} samples ({y_train.sum()} PD, {len(y_train)-y_train.sum()} Normal)")
print(f"   Test:  {X_test.shape[0]} samples ({y_test.sum()} PD, {len(y_test)-y_test.sum()} Normal)")
print(f"\n   PD prevalence in train: {y_train.mean()*100:.2f}%")
print(f"   PD prevalence in test:  {y_test.mean()*100:.2f}%")

# ============================================================================
# 4. BUILD PIPELINE (EXACT SAME AS ORIGINAL)
# ============================================================================

print("\n" + "="*80)
print("4. BUILDING MODEL PIPELINE")
print("="*80)

print("\n🏗️  Pipeline components:")
print("   1. StandardScaler (normalize features)")
print("   2. GradientBoostingClassifier")

print(f"\n⚙️  Model hyperparameters:")
for param, value in MODEL_PARAMS.items():
    print(f"   {param:20s}: {value}")

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', GradientBoostingClassifier(**MODEL_PARAMS))
])

print("\n✅ Pipeline created successfully")

# ============================================================================
# 5. TRAIN MODEL (WITH SAMPLE WEIGHTS - EXACT SAME AS ORIGINAL)
# ============================================================================

print("\n" + "="*80)
print("5. TRAINING MODEL")
print("="*80)

print("\n⚖️  Computing sample weights (balanced)...")
sample_weights = compute_sample_weight('balanced', y_train)
print(f"   Sample weights computed for {len(sample_weights)} training samples")

print("\n🎯 Training GradientBoostingClassifier...")
print("   (This may take 30-60 seconds...)")

pipeline.fit(X_train, y_train, classifier__sample_weight=sample_weights)

print("\n✅ Model trained successfully!")

# ============================================================================
# 6. PREDICTIONS
# ============================================================================

print("\n" + "="*80)
print("6. GENERATING PREDICTIONS")
print("="*80)

# Predictions on test set
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

print("\n✅ Predictions generated for test set")
print(f"   Binary predictions: {len(y_pred)}")
print(f"   Probability scores: {len(y_proba)}")

# ============================================================================
# 7. PERFORMANCE EVALUATION
# ============================================================================

print("\n" + "="*80)
print("7. PERFORMANCE EVALUATION")
print("="*80)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print("\n📊 CLASSIFICATION METRICS:")
print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"   Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"   F1-Score:  {f1:.4f}")
print(f"   ROC-AUC:   {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("\n📊 CONFUSION MATRIX:")
print(f"   True Negatives:  {tn}")
print(f"   False Positives: {fp}")
print(f"   False Negatives: {fn}")
print(f"   True Positives:  {tp}")

# Classification Report
print("\n📊 DETAILED CLASSIFICATION REPORT:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Panic Disorder'], digits=4))

# Probability distribution analysis
print("\n📊 PROBABILITY DISTRIBUTION:")
print(f"   Min probability:    {y_proba.min():.4f}")
print(f"   Max probability:    {y_proba.max():.4f}")
print(f"   Mean probability:   {y_proba.mean():.4f}")
print(f"   Median probability: {np.median(y_proba):.4f}")

# Count predictions in different probability ranges
print("\n📊 PROBABILITY RANGE DISTRIBUTION:")
ranges = [
    (0, 0.01, "< 1%"),
    (0.01, 0.10, "1-10%"),
    (0.10, 0.50, "10-50%"),
    (0.50, 0.90, "50-90%"),
    (0.90, 0.99, "90-99%"),
    (0.99, 1.01, "> 99%")
]

for low, high, label in ranges:
    count = np.sum((y_proba >= low) & (y_proba < high))
    pct = (count / len(y_proba)) * 100
    print(f"   {label:10s}: {count:4d} cases ({pct:5.2f}%)")

# Check for bimodal distribution (like original)
borderline = np.sum((y_proba >= 0.10) & (y_proba <= 0.90))
extreme = len(y_proba) - borderline
print(f"\n🎯 DECISION CONFIDENCE:")
print(f"   Extreme predictions (< 10% or > 90%): {extreme} ({extreme/len(y_proba)*100:.2f}%)")
print(f"   Borderline (10-90%): {borderline} ({borderline/len(y_proba)*100:.2f}%)")

if borderline == 0:
    print("\n   🎉 NO BORDERLINE CASES - Perfect separation maintained!")
elif borderline < 10:
    print(f"\n   ✅ Very few borderline cases - Strong separation!")
else:
    print(f"\n   ⚠️  Significant borderline cases - Separation reduced from original")

# ============================================================================
# 8. CRITICAL ASSESSMENT
# ============================================================================

print("\n" + "="*80)
print("8. CRITICAL ASSESSMENT")
print("="*80)

print("\n🔍 COMPARISON WITH ORIGINAL MODEL (12 features with WHQ_WHD050):")
print("\n   Original model metrics:")
print("   - Expected accuracy: ~100% (reported in Phase 1)")
print("   - Expected borderline: 0 cases")
print("   - Expected ROC-AUC: 1.000")

print(f"\n   New model metrics (11 clean features):")
print(f"   - Accuracy:   {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   - Borderline: {borderline} cases")
print(f"   - ROC-AUC:    {roc_auc:.4f}")

# Determine outcome
if accuracy >= 0.99 and borderline <= 1:
    print("\n   ✅ RESULT: 100% ACCURACY MAINTAINED!")
    print("   🎉 The phenomenon is REAL and not dependent on WHQ_WHD050!")
    print("   ✓  The 11 clean features alone achieve perfect separation")
    print("   ✓  Original findings are VALIDATED with clean data")
    outcome = "SUCCESS"
elif accuracy >= 0.95:
    print("\n   ⚠️  RESULT: Near-perfect accuracy maintained")
    print(f"   → Slight decrease from original (~{(1-accuracy)*100:.1f}% error rate)")
    print("   → WHQ_WHD050 contributed but was not critical")
    print("   → Findings remain strong with clean data")
    outcome = "STRONG"
elif accuracy >= 0.85:
    print("\n   ⚠️  RESULT: Good accuracy but reduced from original")
    print(f"   → Accuracy dropped to {accuracy*100:.1f}%")
    print("   → WHQ_WHD050 was moderately important")
    print("   → Findings need re-interpretation")
    outcome = "MODERATE"
else:
    print("\n   ❌ RESULT: Significant performance degradation")
    print(f"   → Accuracy dropped to {accuracy*100:.1f}%")
    print("   → WHQ_WHD050 was critical to original results")
    print("   → Original 100% accuracy was partially artifact-driven")
    outcome = "DEGRADED"

# ============================================================================
# 9. FEATURE IMPORTANCE
# ============================================================================

print("\n" + "="*80)
print("9. FEATURE IMPORTANCE")
print("="*80)

# Get feature importances from trained model
feature_importances = pipeline.named_steps['classifier'].feature_importances_

# Create dataframe
importance_df = pd.DataFrame({
    'Feature': FEATURES_CLEAN,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

print("\n📊 Top 10 Most Important Features:")
for i, row in importance_df.head(10).iterrows():
    print(f"   {i+1:2d}. {row['Feature']:20s}: {row['Importance']:.4f} ({row['Importance']*100:.2f}%)")

# Check if any single feature is dominant
max_importance = importance_df['Importance'].max()
if max_importance > 0.5:
    print(f"\n   ⚠️  WARNING: Single feature dominance detected ({max_importance*100:.1f}%)")
else:
    print(f"\n   ✅ No single feature dominance (max = {max_importance*100:.1f}%)")

# ============================================================================
# 10. VISUALIZATIONS
# ============================================================================

print("\n" + "="*80)
print("10. GENERATING VISUALIZATIONS")
print("="*80)

# Figure 1: ROC Curve
print("\n📊 Creating ROC curve...")
fig, ax = plt.subplots(figsize=(10, 8))

fpr, tpr, thresholds = roc_curve(y_test, y_proba)
ax.plot(fpr, tpr, linewidth=2.5, label=f'Model (AUC = {roc_auc:.4f})', color='#2E86AB')
ax.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random', alpha=0.5)

ax.set_xlabel('False Positive Rate', fontsize=13, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=13, fontweight='bold')
ax.set_title('ROC Curve: 11 Clean Features Model\n(WHQ_WHD050 removed)', 
             fontsize=15, fontweight='bold', pad=20)
ax.legend(loc='lower right', fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

# Add text box with key metrics
textstr = f'Accuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1-Score: {f1:.4f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
ax.text(0.6, 0.2, textstr, fontsize=11, verticalalignment='top', 
        bbox=props, family='monospace')

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure1_ROC_curve_11features.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"   ✅ Saved: {output_fig}")
plt.close()

# Figure 2: Probability Distribution
print("\n📊 Creating probability distribution plot...")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Overall distribution
ax1.hist(y_proba[y_test == 0], bins=50, alpha=0.6, color='#4ECDC4', 
         label='Normal', edgecolor='black', linewidth=1.2)
ax1.hist(y_proba[y_test == 1], bins=30, alpha=0.6, color='#FF6B6B', 
         label='Panic Disorder', edgecolor='black', linewidth=1.2)
ax1.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold')
ax1.set_xlabel('Predicted Probability', fontsize=13, fontweight='bold')
ax1.set_ylabel('Frequency', fontsize=13, fontweight='bold')
ax1.set_title('Probability Distribution by True Class', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11, frameon=True, shadow=True)
ax1.grid(True, alpha=0.3, axis='y')

# Subplot 2: Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2, 
            xticklabels=['Normal', 'PD'], yticklabels=['Normal', 'PD'],
            cbar_kws={'label': 'Count'}, linewidths=2, linecolor='black')
ax2.set_xlabel('Predicted', fontsize=13, fontweight='bold')
ax2.set_ylabel('True', fontsize=13, fontweight='bold')
ax2.set_title('Confusion Matrix', fontsize=14, fontweight='bold')

plt.suptitle('Model Performance Analysis: 11 Clean Features', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure2_probability_distribution_confusion.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"   ✅ Saved: {output_fig}")
plt.close()

# Figure 3: Feature Importance
print("\n📊 Creating feature importance plot...")
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(importance_df)))
bars = ax.barh(range(len(importance_df)), importance_df['Importance'], 
               color=colors, edgecolor='black', linewidth=1.2)

# Add value labels
for i, (idx, row) in enumerate(importance_df.iterrows()):
    ax.text(row['Importance'] + 0.005, i, f"{row['Importance']:.4f}", 
            va='center', fontsize=10, fontweight='bold')

ax.set_yticks(range(len(importance_df)))
ax.set_yticklabels(importance_df['Feature'], fontsize=11)
ax.set_xlabel('Importance', fontsize=13, fontweight='bold')
ax.set_title('Feature Importance: 11 Clean Features Model\n(WHQ_WHD050 removed)', 
             fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure3_feature_importance_11features.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"   ✅ Saved: {output_fig}")
plt.close()

# Figure 4: Comparison scatter plot (predicted vs true)
print("\n📊 Creating prediction scatter plot...")
fig, ax = plt.subplots(figsize=(10, 8))

# Add jitter for visibility
np.random.seed(42)
jitter_x = np.random.normal(0, 0.02, len(y_test))
jitter_y = np.random.normal(0, 0.02, len(y_test))

scatter = ax.scatter(y_test + jitter_x, y_proba + jitter_y, 
                    c=y_proba, cmap='RdYlGn_r', s=50, alpha=0.6, 
                    edgecolors='black', linewidths=0.5)

ax.set_xlabel('True Label (with jitter)', fontsize=13, fontweight='bold')
ax.set_ylabel('Predicted Probability', fontsize=13, fontweight='bold')
ax.set_title('Predicted Probability vs True Label\n(11 Clean Features)', 
             fontsize=15, fontweight='bold', pad=20)
ax.set_xticks([0, 1])
ax.set_xticklabels(['Normal', 'Panic Disorder'])
ax.axhline(y=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

plt.colorbar(scatter, ax=ax, label='Predicted Probability')
plt.tight_layout()
output_fig = OUTPUT_DIR / 'Figure4_prediction_scatter.png'
plt.savefig(output_fig, dpi=300, bbox_inches='tight')
print(f"   ✅ Saved: {output_fig}")
plt.close()

# ============================================================================
# 11. SAVE RESULTS
# ============================================================================

print("\n" + "="*80)
print("11. SAVING RESULTS")
print("="*80)

# Save model
model_path = OUTPUT_DIR / 'panic_model_11features_CLEAN.joblib'
joblib.dump(pipeline, model_path)
print(f"\n✅ Model saved: {model_path}")

# Save predictions
results_df = pd.DataFrame({
    'True_Label': y_test,
    'Predicted_Label': y_pred,
    'Predicted_Probability': y_proba
})
results_path = OUTPUT_DIR / 'predictions_11features.csv'
results_df.to_csv(results_path, index=False)
print(f"✅ Predictions saved: {results_path}")

# Save feature importance
importance_path = OUTPUT_DIR / 'feature_importance_11features.csv'
importance_df.to_csv(importance_path, index=False)
print(f"✅ Feature importance saved: {importance_path}")

# Save metrics summary
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC',
               'True Negatives', 'False Positives', 'False Negatives', 'True Positives',
               'Borderline Cases', 'Extreme Cases'],
    'Value': [accuracy, precision, recall, f1, roc_auc,
              tn, fp, fn, tp, borderline, extreme]
})
metrics_path = OUTPUT_DIR / 'metrics_summary_11features.csv'
metrics_df.to_csv(metrics_path, index=False)
print(f"✅ Metrics saved: {metrics_path}")

# ============================================================================
# 12. FINAL REPORT
# ============================================================================

print("\n" + "="*80)
print("12. GENERATING FINAL REPORT")
print("="*80)

report = f"""
MODEL RE-TRAINING REPORT: 11 CLEAN FEATURES
============================================

Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

OBJECTIVE
---------
Test if 100% accuracy is maintained after removing contaminated feature WHQ_WHD050
(which contained 99999 coded values creating 96.49% single-variable discrimination).

DATASET
-------
- Source: NHANES_panic_11features_CLEAN.csv
- Total samples: {len(df_processed)}
- Features: {len(FEATURES_CLEAN)} (WHQ_WHD050 removed)
- PD prevalence: {prevalence*100:.2f}%

TRAIN/TEST SPLIT
----------------
- Training: {len(X_train)} samples ({y_train.sum()} PD, {len(y_train)-y_train.sum()} Normal)
- Testing: {len(X_test)} samples ({y_test.sum()} PD, {len(y_test)-y_test.sum()} Normal)
- Split ratio: {(1-TEST_SIZE)*100:.0f}/{TEST_SIZE*100:.0f}
- Random state: {RANDOM_STATE}

MODEL CONFIGURATION
-------------------
Algorithm: Gradient Boosting Classifier
Hyperparameters (EXACT same as original):
  - n_estimators: {MODEL_PARAMS['n_estimators']}
  - max_depth: {MODEL_PARAMS['max_depth']}
  - learning_rate: {MODEL_PARAMS['learning_rate']}
  - min_samples_leaf: {MODEL_PARAMS['min_samples_leaf']}
  - min_samples_split: {MODEL_PARAMS['min_samples_split']}
  - random_state: {MODEL_PARAMS['random_state']}

Preprocessing:
  - StandardScaler (feature normalization)
  - SimpleImputer with median strategy
  - Balanced sample weights

PERFORMANCE METRICS
-------------------
Accuracy:     {accuracy:.4f} ({accuracy*100:.2f}%)
Precision:    {precision:.4f} ({precision*100:.2f}%)
Recall:       {recall:.4f} ({recall*100:.2f}%)
F1-Score:     {f1:.4f}
ROC-AUC:      {roc_auc:.4f}

CONFUSION MATRIX
----------------
                Predicted
                Normal    PD
Actual Normal     {tn:4d}    {fp:4d}
Actual PD         {fn:4d}    {tp:4d}

DECISION CONFIDENCE
-------------------
Extreme predictions (< 10% or > 90%): {extreme} ({extreme/len(y_proba)*100:.2f}%)
Borderline (10-90%): {borderline} ({borderline/len(y_proba)*100:.2f}%)

TOP 5 MOST IMPORTANT FEATURES
------------------------------
"""

for i, row in importance_df.head(5).iterrows():
    report += f"{i+1}. {row['Feature']:20s}: {row['Importance']:.4f} ({row['Importance']*100:.2f}%)\n"

report += f"""

CRITICAL ASSESSMENT
-------------------
Outcome: {outcome}

"""

if outcome == "SUCCESS":
    report += """
✅ THE PHENOMENON IS REAL AND VALIDATED!

The model achieves 100% accuracy with 11 clean features alone, demonstrating that:
1. The perfect separation is NOT dependent on the contaminated WHQ_WHD050
2. The 11 remaining features contain sufficient discriminative power
3. The original findings are VALID and can be trusted
4. The synergistic feature interactions hypothesis is strongly supported

NEXT STEPS:
- Proceed with all planned analyses (SHAP, UMAP, Decision Trees)
- Continue with Paper 3 investigation
- Original research conclusions are VALIDATED
"""
elif outcome == "STRONG":
    report += f"""
⚠️  NEAR-PERFECT PERFORMANCE MAINTAINED

The model achieves {accuracy*100:.1f}% accuracy with 11 clean features:
1. Performance slightly reduced but remains excellent
2. WHQ_WHD050 contributed but was not critical
3. Findings remain strong and publishable
4. Minor adjustments to paper language needed

NEXT STEPS:
- Proceed with analyses but note the small performance difference
- Update paper to report {accuracy*100:.1f}% accuracy with clean data
- Emphasize that contaminated feature was detected and removed
"""
elif outcome == "MODERATE":
    report += f"""
⚠️  MODERATE PERFORMANCE MAINTAINED

The model achieves {accuracy*100:.1f}% accuracy with 11 clean features:
1. Significant reduction from original 100%
2. WHQ_WHD050 was moderately important
3. Findings need re-interpretation
4. Paper focus should shift from "perfect" to "high accuracy"

NEXT STEPS:
- Continue analyses but adjust expectations
- Re-frame paper around {accuracy*100:.1f}% accuracy
- Investigate why WHQ_WHD050 was important (beyond the 99999 codes)
- Consider ensemble with other features
"""
else:
    report += f"""
❌ SIGNIFICANT PERFORMANCE DEGRADATION

The model achieves only {accuracy*100:.1f}% accuracy with 11 clean features:
1. Major reduction from original 100%
2. WHQ_WHD050 was critical to original results
3. Original 100% accuracy was substantially artifact-driven
4. Project requires major re-evaluation

NEXT STEPS:
- Re-evaluate whether to continue with current approach
- Consider alternative modeling strategies
- May need to pivot Paper 3 focus to data quality issues
- Thoroughly investigate remaining features for other artifacts
"""

report += f"""

TECHNICAL DETAILS
-----------------
All files saved to: {OUTPUT_DIR}

Generated files:
- panic_model_11features_CLEAN.joblib (trained model)
- predictions_11features.csv (test set predictions)
- feature_importance_11features.csv (feature rankings)
- metrics_summary_11features.csv (performance metrics)
- Figure1_ROC_curve_11features.png
- Figure2_probability_distribution_confusion.png
- Figure3_feature_importance_11features.png
- Figure4_prediction_scatter.png

REPRODUCIBILITY
---------------
Random state: {RANDOM_STATE} (fixed for reproducibility)
All hyperparameters identical to original model
Same preprocessing pipeline
Same train/test split strategy

Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

report_path = OUTPUT_DIR / 'MODEL_RETRAIN_REPORT.txt'
with open(report_path, 'w') as f:
    f.write(report)
print(f"\n✅ Report saved: {report_path}")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("✅ MODEL RE-TRAINING COMPLETE!")
print("="*80)

print(f"\n🎯 FINAL RESULT: {outcome}")
print(f"\n📊 Key Metrics:")
print(f"   Accuracy:  {accuracy*100:.2f}%")
print(f"   ROC-AUC:   {roc_auc:.4f}")
print(f"   Borderline: {borderline} cases")

if outcome == "SUCCESS":
    print("\n🎉 THE 100% ACCURACY IS REAL!")
    print("   ✓ Findings validated with clean data")
    print("   ✓ Continue with Paper 3 investigations")
    print("   ✓ WHQ_WHD050 was not critical")
elif outcome == "STRONG":
    print("\n✅ Near-perfect accuracy maintained!")
    print("   ✓ Findings remain strong")
    print("   ✓ Minor adjustments to paper needed")
elif outcome == "MODERATE":
    print("\n⚠️  Accuracy reduced but still good")
    print("   → Re-frame paper expectations")
    print("   → Continue with adjusted approach")
else:
    print("\n❌ Significant accuracy drop")
    print("   → Original results partly artifact-driven")
    print("   → Major re-evaluation needed")

print(f"\n📂 All results saved to: {OUTPUT_DIR}")
print("\n" + "="*80)

🤖 RE-TRAINING MODEL WITH 11 CLEAN FEATURES

📂 Data: /Users/filipecarvalho/Documents/data_science_projects/Panic.3/NHANES_panic_11features_CLEAN.csv
📂 Output: /Users/filipecarvalho/Documents/data_science_projects/Panic.3/results/model_retrain_11features

📋 Using 11 clean features
🎲 Random state: 42
📊 Test size: 25.0%

1. LOADING CLEAN DATASET

✅ Loaded: 6581 samples × 12 columns
✅ All 11 features present

🎯 Target distribution:
   Normal          (target=0):  6403 (97.30%)
   Panic Disorder  (target=1):   178 ( 2.70%)

📊 PD Prevalence: 2.70%

2. PREPROCESSING

🔧 Imputing missing values with median (same as original)...
   Imputed BPX_BPXDAR
   Imputed DEMO_INDFMPIR
   Imputed ALQ_ALQ130
   Imputed BMX_BMXBMI
   Imputed DEMO_DMDMARTL
   Imputed BIX_BIDFAT

✅ No missing values after imputation

✅ Final dataset:
   X shape: (6581, 11)
   y shape: (6581,)
   Features: 11

3. TRAIN/TEST SPLIT

📊 Split summary:
   Train: 4935 samples (133 PD, 4802 Normal)
   Test:  1646 samples (45 PD, 1601 N